In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESTSF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESTSF;
using XDGShock;
Init();


In [2]:
var ass= new MatrixAssembler(1);
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESTSF_AcousticWave1D_WaveLengths'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESTSF_AcousticWave1D_WaveLengths'.


{ Session Count = 27; Grid Count = 27; Path = \\dc3\userspace\sebastian\cluster\XESTSF_AcousticWave1D_WaveLengths }

In [3]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

\\dc3\userspace\sebastian\cluster\XESTSF_AcousticWave1D_WaveLengths

In [4]:
(double[] xVals, double[]yVals) GetPlot(double xMin,double xMax, double t, int nRef,string field,ISessionInfo si){
//double shockPosition=0.5;
var p1= new double[] {xMin,t};
var p2=new double[] {xMax,t};

//timesteps to compare
var tiEnd=si.Timesteps.Last();

// # use p_per from BoSSS 
var pEnd =tiEnd.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

In [5]:
var db=OpenOrCreateDatabase(dbPath);
var ses=db.Sessions;
ses

#0: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.15	2/28/2024 1:49:17 PM	7a779b5a...
#1: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.08	2/28/2024 1:50:03 PM	8cd963ca...
#2: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.05	2/28/2024 1:49:40 PM	fe4eb8c4...
#3: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.01	2/28/2024 1:50:28 PM	55dd41fc...
#4: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.2	2/28/2024 1:48:32 PM	48fe4264...
#5: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.1	2/28/2024 1:48:53 PM	477a368c...
#6: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_1s

In [84]:
var si=db.Sessions.Pick(8);
var ti=si.Timesteps.Last();
//si.Timesteps
si

XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.15	2/28/2024 1:51:47 PM	93e7120e...

In [85]:
si.Timesteps.Count

41

# Plotting

define general params for plotting (must substract some eps because, otherwise some points are not located in the space-time grid)

In [86]:
var eps=0.000001;
double xMin=0 +eps;double xMax=3.0-eps; int xRef=200; 
double tMin =0 + eps; double tMax=1.5-eps;int tRef=6;
int nRows=2;

In [87]:
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;

## Pressure Pertubations

In [88]:
string field="p_per";
string csvFileName = "XESTSFPressurePertubations.csv";
Plot2Ddata[,] multiplotsPressure = new Plot2Ddata[(tRef+1)/nRows,nRows];
using (StreamWriter writer = new StreamWriter(csvFileName))
        {
        for(int Ti=0;Ti< tRef;Ti++){
            var plot= new Plot2Ddata();
            plot.Xlabel="x";
            plot.Ylabel="p'";
            double t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
            //Console.WriteLine(t);
            (double[] xVals,double[] yVals)=GetPlot(xMin,xMax,t,xRef,field, si);
            plot.AddDataGroup("t="+Math.Round(t,2),xVals,yVals,fmt);
            multiplotsPressure[Ti/nRows,Ti %nRows]=plot;
            if (Ti == 0)
            {
                // Write header row
                writer.WriteLine($"x/t,{string.Join(",", xVals)}");
            }
            // Write data row
            writer.WriteLine($"{Math.Round(t, 2)},{string.Join(",", yVals)}");
                    
        }
};


In [89]:
Array.ForEach(multiplotsPressure.Cast<Plot2Ddata>().ToArray(), 
    plot => { plot.YrangeMin = -1.5 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });
// multiplotsPressure[1,1].YrangeMax = 5 * 1e-7; multiplotsPressure[1,1].YrangeMin = -6 * 1e-7;
// multiplotsPressure[2,0].YrangeMax = 5 * 1e-7; multiplotsPressure[2,0].YrangeMin = -6 * 1e-7;
// multiplotsPressure[2,1].YrangeMax = 5 * 1e-7; multiplotsPressure[2,1].YrangeMin = -6 * 1e-7;
multiplotsPressure.ToGnuplot().PlotSVG(xRes:900,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.5x10 -5 
 
 
 
 
 -1x10 -5 
 
 
 
 
 -5x10 -6 
 
 
 
 
 0 
 
 
 
 
 5x10 -6 
 
 
 
 
 1x10 -5 
 
 
 
 
 1.5x10 -5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 p' 
 
 
 
 
 x 
 
 
 
 
 t=0 
 
 
 t=0 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M289.5,59.1 L342.9,59.1 M109.5,86.5 L110.9,73.2 L112.4,59.3 L113.8,48.4 L115.3,65.3 L116.7,85.9
 L118.1,114.6 L119.6,125.2 L121.0,125.6 L122.4,110.0 L123.9,90.9 L125.3,83.8 L126.8,84.8 L128.2,96.6
 L129.6,91.2 L131.1,91.0 L132.5,88.7 L133.9,88.9 L135.4,88.9 L136.8,89.0 L138.3,89.3 L139.7,89.3
 L141.1,89.3 L142.6,89.3 L144.0,89.3 L145.4,89.2 L146.9,89.2 L148.3,89.2 L149.8,89.2 L151.2,89.3
 L152.6,89.3 L154.1,89.3 L155.5,89.2 L156.9,89.2 L158.4,89.2 L159.8,89.2 L161.3,89.3 L162.7,89.3
 L164.1,89.3 L165.6,89.2 L167.0,89.2 L168.4,89.2 L169.9,89.3 L171.3,89.3 L172.8,89.3 L174.2,89.3
 L175.6,89.2 L177.1,89.2 L178.5,89.2 L179.9,89.3 L181.4,89.3 L182.8,89.3 L184.3,89.2 L185.7,89.2
 L187.1,89.2 L188.6,89.3 L190.0,89.3 L191.4,89.3 L192.9,89.3 L194.3,89.3 L195.8,89.3 L197.2,89.3
 L198.6,89.3 L200.1,89.3 L201.5,89.3 L202.9,89.3 L204.4,89.3 L205.8,89.3 L207.3,89.3 L208.7,89.3
 L210.1,89.3 L211.6,89.3 L213.0,89.3 L214.5,89.3 L215.9,89.3 L217.3,89.3 L218.8,89.3 L220.2,89.3
 L221.6,89.3 L223.1,89.3 L224.5,89.3 L226.0,89.3 L227.4,89.3 L228.8,89.3 L230.3,89.3 L231.7,89.3
 L233.1,89.3 L234.6,89.3 L236.0,89.3 L237.5,89.3 L238.9,89.3 L240.3,89.3 L241.8,89.2 L243.2,89.3
 L244.6,89.3 L246.1,89.2 L247.5,89.2 L249.0,89.3 L250.4,89.3 L251.8,89.4 L253.3,89.4 L254.7,89.3
 L256.1,89.2 L257.6,89.2 L259.0,89.3 L260.5,89.3 L261.9,89.3 L263.3,89.2 L264.8,89.2 L266.2,89.2
 L267.6,89.3 L269.1,89.3 L270.5,89.3 L272.0,89.3 L273.4,89.2 L274.8,89.2 L276.3,89.2 L277.7,89.3
 L279.1,89.3 L280.6,89.3 L282.0,89.2 L283.5,89.2 L284.9,89.2 L286.3,89.3 L287.8,89.3 L289.2,89.3
 L290.6,89.3 L292.1,89.2 L293.5,89.2 L295.0,89.3 L296.4,89.3 L297.8,89.3 L299.3,89.3 L300.7,89.3
 L302.2,89.3 L303.6,89.3 L305.0,89.3 L306.5,89.3 L307.9,89.3 L309.3,89.3 L310.8,89.3 L312.2,89.3
 L313.7,89.3 L315.1,89.3 L316.5,89.3 L318.0,89.3 L319.4,89.3 L320.8,89.3 L322.3,89.3 L323.7,89.3
 L325.2,89.3 L326.6,89.3 L328.0,89.3 L329.5,89.3 L330.9,89.3 L332.3,89.3 L333.8,89.3 L335.2,89.3
 L336.7,89.3 L338.1,89.3 L339.5,89.3 L341.0,89.3 L342.4,89.3 L343.8,89.3 L345.3,89.3 L346.7,89.3
 L348.2,89.3 L349.6,89.3 L351.0,89.3 L352.5,89.3 L353.9,89.3 L355.3,89.3 L356.8,89.3 L358.2,89.3
 L359.7,89.3 L361.1,89.3 L362.5,89.3 L364.0,89.3 L365.4,89.3 L366.8,89.3 L368.3,89.3 L369.7,89.3
 L371.2,89.3 L372.6,89.3 L374.0,89.3 L375.5,89.3 L376.9,89.3 L378.3,89.3 L379.8,89.3 L381.2,89.3
 L382.7,89.3 L384.1,89.3 L385.5,89.3 L387.0,89.3 L388.4,89.3 L389.8,89.3 L391.3,89.3 L392.7,89.3
 L394.2,89.3 L395.6,89.3 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.5x10 -5 
 
 
 
 
 -1x10 -5 
 
 
 
 
 -5x10 -6 
 
 
 
 
 0 
 
 
 
 
 5x10 -6 
 
 
 
 
 1x10 -5 
 
 
 
 
 1.5x10 -5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 p' 
 
 
 
 
 x 
 
 
 
 
 t=0.3 
 
 
 t=0.3 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M717.3,59.1 L770.7,59.1 M559.5,89.3 L560.9,89.3 L562.4,89.2 L563.8,89.2 L565.3,89.2 L566.7,89.3
 L568.1,89.3 L569.6,89.3 L571.0,89.3 L572.4,89.2 L573.9,89.3 L575.3,89.2 L576.8,89.3 L578.2,89.3
 L579.6,89.3 L581.1,89.3 L582.5,89.3 L583.9,89.3 L585.4,89.3 L586.8,89.2 L588.3,89.2 L589.7,89.2
 L591.1,89.2 L592.6,89.2 L594.0,89.3 L595.4,89.3 L596.9,

## Density Pertubations

In [90]:
string field="rho_per";
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;

Plot2Ddata[,] multiplotsDensity = new Plot2Ddata[(tRef+1)/nRows,nRows];
string csvFileName = "XESTSFDensityPertubations.csv";
using (StreamWriter writer = new StreamWriter(csvFileName)){
    for(int Ti=0;Ti< tRef;Ti++){
        var plot= new Plot2Ddata();
        plot.Xlabel="x";
        plot.Ylabel="rho'";
        double t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
        //Console.WriteLine(t);
        (double[] xVals,double[] yVals)=GetPlot(xMin,xMax,t,xRef,field, si);
        plot.AddDataGroup("t="+Math.Round(t,2),xVals,yVals,fmt);
        multiplotsDensity[Ti/nRows,Ti %nRows]=plot;
        if (Ti == 0)
        {
            // Write header row
            writer.WriteLine($"x/t,{string.Join(",", xVals)}");
        }
        // Write data row
        writer.WriteLine($"{Math.Round(t, 2)},{string.Join(",", yVals)}");
                
    }
    
};


In [91]:
Array.ForEach(multiplotsDensity.Cast<Plot2Ddata>().ToArray(), 
    plot => { plot.YrangeMin = -1 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });
// multiplotsDensity[1,1].YrangeMax = 5 * 1e-7; multiplotsDensity[1,1].YrangeMin = -6 * 1e-7;
//multiplotsDensity[2,0].YrangeMax = 5 * 1e-7; multiplotsDensity[2,0].YrangeMin = -6 * 1e-7;
//multiplotsDensity[2,1].YrangeMax = 5 * 1e-7; multiplotsDensity[2,1].YrangeMin = -6 * 1e-7;
multiplotsDensity.ToGnuplot().PlotSVG(xRes:1000,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1x10 -5 
 
 
 
 
 -5x10 -6 
 
 
 
 
 0 
 
 
 
 
 5x10 -6 
 
 
 
 
 1x10 -5 
 
 
 
 
 1.5x10 -5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 rho' 
 
 
 
 
 x 
 
 
 
 
 t=0 
 
 
 t=0 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M339.0,59.1 L392.4,59.1 M110.0,97.1 L111.7,85.8 L113.4,73.8 L115.1,64.5 L116.7,79.0 L118.4,96.6
 L120.1,121.2 L121.8,130.3 L123.5,130.6 L125.2,117.2 L126.8,100.9 L128.5,94.8 L130.2,95.6 L131.9,105.8
 L133.6,101.2 L135.3,101.0 L136.9,99.0 L138.6,99.2 L140.3,99.1 L142.0,99.3 L143.7,99.5 L145.4,99.5
 L147.0,99.5 L148.7,99.5 L150.4,99.5 L152.1,99.5 L153.8,99.5 L155.5,99.5 L157.1,99.5 L158.8,99.5
 L160.5,99.5 L162.2,99.5 L163.9,99.5 L165.6,99.5 L167.3,99.5 L168.9,99.5 L170.6,99.5 L172.3,99.5
 L174.0,99.5 L175.7,99.5 L177.4,99.5 L179.0,99.5 L180.7,99.5 L182.4,99.5 L184.1,99.5 L185.8,99.5
 L187.5,99.5 L189.1,99.5 L190.8,99.5 L192.5,99.5 L194.2,99.5 L195.9,99.5 L197.6,99.5 L199.2,99.5
 L200.9,99.5 L202.6,99.5 L204.3,99.5 L206.0,99.5 L207.7,99.5 L209.4,99.5 L211.0,99.5 L212.7,99.5
 L214.4,99.5 L216.1,99.5 L217.8,99.5 L219.5,99.5 L221.1,99.5 L222.8,99.5 L224.5,99.5 L226.2,99.5
 L227.9,99.5 L229.6,99.5 L231.2,99.5 L232.9,99.5 L234.6,99.5 L236.3,99.5 L238.0,99.5 L239.7,99.5
 L241.3,99.5 L243.0,99.5 L244.7,99.5 L246.4,99.5 L248.1,99.5 L249.8,99.5 L251.4,99.5 L253.1,99.5
 L254.8,99.5 L256.5,99.5 L258.2,99.5 L259.9,99.5 L261.6,99.5 L263.2,99.5 L264.9,99.5 L266.6,99.5
 L268.3,99.5 L270.0,99.5 L271.7,99.5 L273.3,99.5 L275.0,99.5 L276.7,99.6 L278.4,99.6 L280.1,99.5
 L281.8,99.5 L283.4,99.5 L285.1,99.5 L286.8,99.5 L288.5,99.5 L290.2,99.4 L291.9,99.4 L293.5,99.4
 L295.2,99.5 L296.9,99.7 L298.6,99.6 L300.3,99.4 L302.0,99.2 L303.7,99.3 L305.3,99.5 L307.0,99.8
 L308.7,99.7 L310.4,99.5 L312.1,99.3 L313.8,99.2 L315.4,99.4 L317.1,99.6 L318.8,99.7 L320.5,99.6
 L322.2,99.4 L323.9,99.3 L325.5,99.4 L327.2,99.5 L328.9,99.5 L330.6,99.5 L332.3,99.5 L334.0,99.5
 L335.6,99.5 L337.3,99.5 L339.0,99.5 L340.7,99.4 L342.4,99.5 L344.1,99.5 L345.7,99.5 L347.4,99.5
 L349.1,99.5 L350.8,99.4 L352.5,99.5 L354.2,99.5 L355.9,99.5 L357.5,99.5 L359.2,99.5 L360.9,99.5
 L362.6,99.5 L364.3,99.5 L366.0,99.5 L367.6,99.5 L369.3,99.5 L371.0,99.5 L372.7,99.5 L374.4,99.5
 L376.1,99.5 L377.7,99.5 L379.4,99.5 L381.1,99.5 L382.8,99.5 L384.5,99.5 L386.2,99.5 L387.8,99.5
 L389.5,99.5 L391.2,99.5 L392.9,99.5 L394.6,99.5 L396.3,99.5 L398.0,99.5 L399.6,99.5 L401.3,99.5
 L403.0,99.5 L404.7,99.5 L406.4,99.5 L408.1,99.5 L409.7,99.5 L411.4,99.5 L413.1,99.5 L414.8,99.5
 L416.5,99.5 L418.2,99.5 L419.8,99.5 L421.5,99.5 L423.2,99.5 L424.9,99.5 L426.6,99.5 L428.3,99.5
 L429.9,99.5 L431.6,99.5 L433.3,99.5 L435.0,99.5 L436.7,99.5 L438.4,99.5 L440.0,99.5 L441.7,99.5
 L443.4,99.5 L445.1,99.5 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1x10 -5 
 
 
 
 
 -5x10 -6 
 
 
 
 
 0 
 
 
 
 
 5x10 -6 
 
 
 
 
 1x10 -5 
 
 
 
 
 1.5x10 -5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 rho' 
 
 
 
 
 x 
 
 
 
 
 t=0.3 
 
 
 t=0.3 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M816.8,59.1 L870.2,59.1 M610.0,99.5 L611.7,99.5 L613.4,99.5 L615.1,99.5 L616.7,99.5 L618.4,99.5
 L620.1,99.5 L621.8,99.5 L623.5,99.5 L625.2,99.5 L626.8,99.5 L628.5,99.5 L630.2,99.5 L631.9,99.5
 L633.6,99.5 L635.3,99.5 L636.9,99.5 L638.6,99.5 L640.3,99.5 L642.0,99.5 L643.7,99.5 L645.4,99.5
 L647.0,99.5 L648.7,99.5 L650.4,99.5 L652.1,99.5 L653.8,99.5 L655.5,99.5 L657.1,99.5 L658.8,99.5
 L660.5,99.5 

In [93]:
(double[] xVals, double[]yVals) GetPlotLevelSet(double tMin,double tMax, int nRef,ISessionInfo si){
//double shockPosition=0.5;
var p1= new double[] {0.0,tMin};
var p2=new double[] {0.0,tMax};

//timesteps to compare
var tiEnd=si.Timesteps.Last();
// # use p_per from BoSSS 
var pEnd =tiEnd.GetField("LevelSet");
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

In [94]:
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;
string csvFileName = "XESTSFLevelSetPertubations.csv";
var plotLevelSet= new Plot2Ddata();
using (StreamWriter writer = new StreamWriter(csvFileName)){
    
    //Console.WriteLine(t);
    (double[] xVals,double[] yVals)=GetPlotLevelSet(tMin,tMax,1000, si);
    
    plotLevelSet.AddDataGroup("LevelSet",xVals,yVals,fmt);
        // Write header row
    writer.WriteLine($"t,{string.Join(",", xVals)}");
    
    // Write data row
    writer.WriteLine($"phi(t),{string.Join(",", yVals)}");
            
}

plotLevelSet.Ylabel="\varphi(0,t)";
plotLevelSet.ToGnuplot().PlotSVG(xRes:800,yRes:800)  


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.5000001 
 
 
 
 
 -1.5000000 
 
 
 
 
 -1.5000000 
 
 
 
 
 -1.5000000 
 
 
 
 
 -1.4999999 
 
 
 
 
 -1.4999999 
 
 
 
 
 -1.4999998 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 arphi(0,t) 
 
 
 
 
 LevelSet 
 
 
 LevelSet 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M588.5,57.1 L641.9,57.1 M121.7,521.4 L122.3,520.6 L123.0,519.9 L123.6,519.2 L124.2,518.4 L124.8,517.7
 L125.5,517.0 L126.1,516.3 L126.7,515.5 L127.4,514.8 L128.0,514.1 L128.6,513.3 L129.2,512.6 L129.9,511.9
 L130.5,511.1 L131.1,510.4 L131.8,509.7 L132.4,509.4 L133.0,509.3 L133.7,509.3 L134.3,509.2 L134.9,509.2
 L135.5,509.1 L136.2,509.1 L136.8,509.0 L137.4,509.0 L138.1,508.9 L138.7,508.9 L139.3,508.8 L139.9,508.8
 L140.6,508.7 L141.2,508.7 L141.8,508.6 L142.5,508.6 L143.1,508.7 L143.7,508.8 L144.3,508.9 L145.0,509.0
 L145.6,509.2 L146.2,509.3 L146.9,509.4 L147.5,509.5 L148.1,509.6 L148.7,509.7 L149.4,509.8 L150.0,509.9
 L150.6,510.0 L151.3,510.1 L151.9,510.3 L152.5,510.4 L153.2,510.2 L153.8,510.1 L154.4,509.9 L155.0,509.8
 L155.7,509.6 L156.3,509.5 L156.9,509.3 L157.6,509.2 L158.2,509.0 L158.8,508.9 L159.4,508.7 L160.1,508.6
 L160.7,508.4 L161.3,508.3 L162.0,508.1 L162.6,508.0 L163.2,508.0 L163.8,508.1 L164.5,508.3 L165.1,508.5
 L165.7,508.7 L166.4,508.9 L167.0,509.0 L167.6,509.2 L168.2,509.4 L168.9,509.6 L169.5,509.7 L170.1,509.9
 L170.8,510.1 L171.4,510.3 L172.0,510.4 L172.7,510.6 L173.3,510.8 L173.9,510.5 L174.5,510.3 L175.2,510.1
 L175.8,509.9 L176.4,509.7 L177.1,509.5 L177.7,509.3 L178.3,509.1 L178.9,508.9 L179.6,508.7 L180.2,508.4
 L180.8,508.2 L181.5,508.0 L182.1,507.8 L182.7,507.6 L183.3,507.4 L184.0,507.5 L184.6,507.8 L185.2,508.1
 L185.9,508.3 L186.5,508.6 L187.1,508.8 L187.7,509.1 L188.4,509.3 L189.0,509.6 L189.6,509.9 L190.3,510.1
 L190.9,510.4 L191.5,510.6 L192.2,510.9 L192.8,511.1 L193.4,511.4 L194.0,511.5 L194.7,511.2 L195.3,510.9
 L195.9,510.6 L196.6,510.3 L197.2,510.0 L197.8,509.7 L198.4,509.4 L199.1,509.1 L199.7,508.8 L200.3,508.5
 L201.0,508.2 L201.6,507.9 L202.2,507.6 L202.8,507.3 L203.5,507.0 L204.1,506.7 L204.7,506.9 L205.4,507.2
 L206.0,507.5 L206.6,507.8 L207.2,508.0 L207.9,508.3 L208.5,508.6 L209.1,508.9 L209.8,509.2 L210.4,509.4
 L211.0,509.7 L211.7,510.0 L212.3,510.3 L212.9,510.6 L213.5,510.8 L214.2,511.1 L214.8,511.2 L215.4,511.1
 L216.1,511.0 L216.7,510.9 L217.3,510.9 L217.9,510.8 L218.6,510.7 L219.2,510.6 L219.8,510.5 L220.5,510.5
 L221.1,510.4 L221.7,510.3 L222.3,510.2 L223.0,510.1 L223.6,510.1 L224.2,510.0 L224.9,509.8 L225.5,509.3
 L226.1,508.9 L226.7,508.4 L227.4,507.9 L228.0,507.4 L228.6,506.9 L229.3,506.5 L229.9,506.0 L230.5,505.5
 L231.2,505.0 L231.8,504.6 L232.4,504.1 L233.0,503.6 L233.7,503.1 L234.3,502.6 L234.9,502.2 L235.6,503.4
 L236.2,505.0 L236.8,506.5 L237.4,508.1 L238.1,509.7 L238.7,511.2 L239.3,512.8 L240.0,514.4 L240.6,515.9
 L241.2,517.5 L241.8,519.0 L242.5,520.6 L243.1,522.2 L243.7,523.7 L244.4,525.3 L245.0,526.8 L245.6,526.2
 L246.2,523.1 L246.9,519.9 L247.5,516.8 L248.1,513.7 L248.8,510.5 L249.4,507.4 L250.0,504.3 L250.7,501.1
 L251.3,498.0 L251.9,494.9 L252.5,491.8 L253.2,488.6 L253.8,485.5 L254.4,482.4 L255.1,479.2 L255.7,476.9
 L256.3,481.5 L256.9,486.2 L257.6,490.8 L258.2,495.5 L258.8,500.2 L259.5,504.8 L260.1,509.5 L260.7,514.1
 L261.3,518.8 L262.0,523.4 L262.6,528.1 L263.2,532.7 L263.9,537.4 L264.5,542.0 L265.1,546.7 L265.7,551.4
 L266.4,549.0 L267.0,544.0 L267.6,539.0 L268.3,533.9 L268.9,528.9 L269.5,523.9 L270.2,518.9 L270.8,513.8
 L271.4,508.8 L272.0,503.8 L272.7,498.8 L273.3,493.7 L273.9,488.7 L274.6,483.7 L275.2,478.7 L275.8,473.6
 